# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [12]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv










#### Creating list of filepaths to process original event csv data files

In [13]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [14]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [15]:
# The number of rows in the csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


#### Creating a Cluster

In [16]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [17]:

try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS casandra
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [18]:

try:
    session.set_keyspace('casandra')
except Exception as e:
    print(e)


In [19]:
#### The table was created belwo to get the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

#### To create the table I used (session_id, iteminsession) as PRIMARY KEY to uniquely identify values. Here session_id is the partition key and iteminsession is the 
## clustering key with other attributes included to answer the query. 

In [20]:
query = "CREATE TABLE IF NOT EXISTS music_app_history"

query = query + "(session_id int, artist text, iteminsession int, length float, song text, PRIMARY KEY(session_id, iteminsession))"
try:
    session.execute(query)
    print('table_created')
except Exception as e:
    print(e)    

table_created


In [21]:

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        query =  "INSERT INTO music_app_history (session_id, artist, iteminsession, length, song)"
        query = query + "VALUES (%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[8]), line[0], int(line[3]), float(line[5]), line[9]))
    

In [22]:


query = "SELECT artist, song, length FROM music_app_history WHERE session_id = 338 AND iteminsession = 4"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


In [23]:
#### The table below was created to give the name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

#### I created a table with (session_id,  userid) set as partition key to uniquely identify values and iteminsession as clustering key to sort the songs. 

In [26]:


query = "CREATE TABLE IF NOT EXISTS song_playlist_session"      
query = query + "( session_id int, userid int, iteminsession int, artist text, firstname text, lastname text, song text, PRIMARY KEY((session_id,  userid), iteminsession))"
try:
    session.execute(query)
    print('table_created')
except Exception as e:
    print(e)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query =  "INSERT INTO song_playlist_session (session_id, userid, iteminsession, artist, firstname, lastname, song)"
        query = query + "VALUES (%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[8]), int(line[10]), int(line[3]), line[0], line[1], line[4], line[9]))


query = "SELECT artist, firstname, lastname, song FROM song_playlist_session WHERE session_id = 182 AND userid = 10 ORDER BY iteminsession"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)

table_created
Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [27]:
#### This table below was modeled after the query which wanted to know the first and last name of a user who listened to the song  'All Hands Against His Own'

#### I created a table with Primary key (song, userid) to uniquely get the names.


In [30]:


query = "CREATE TABLE IF NOT EXISTS user_song_session"
query = query + "(song text, firstname text, lastname text, userid int, PRIMARY KEY(song, userid))"
try:
    session.execute(query)
    #print('table_created')
except Exception as e:
    print(e)


file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query =  "INSERT INTO user_song_session (song, firstname, lastname, userid)"
        query = query + "VALUES (%s,%s,%s, %s)"
        session.execute(query, (line[9], line[1], line[4], int(line[10])))


query = "SELECT firstname, lastname FROM user_song_session WHERE song = 'All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)



                    

Jacqueline Lynch
Tegan Levine
Sara Johnson


In [31]:
session.shutdown()
cluster.shutdown()